In [1]:
# Project Template Import Cell
%matplotlib inline
import os
import sys
import math

from __future__ import print_function, division
from inspect import getsourcefile

from IPython.display import FileLink

# Standard Notebook Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

CURR_PATH = os.path.abspath(getsourcefile(lambda:0)).rsplit('/', 1)[0] # Get filepath of this notebook
module_path = os.path.join(os.path.dirname(CURR_PATH), 'utils') # Make module path for one dir up and one down into utils
if module_path not in sys.path: # Append to system path list
    sys.path.append(module_path)

import utils as utils ; reload(utils)

print('System : Python ', os.sys.version , '\n\n')

print('Directory Structure:')
print(CURR_PATH)
!tree -cn --filelimit 13

# Keras Setup
import keras
print('\n\nKeras version:' , keras.__version__ ,
      ', backend:' , keras.backend.backend(),
      ', image_format:' , keras.backend.image_data_format())

random_seed = 2

print('\n\nEnvironment : ')
FileLink('../../ana_py27_p2.yml')

Using Theano backend.


System : Python  2.7.12 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:42:40) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)] 


Directory Structure:
/media/david/5C14F53A14F517AA/code/ana_py27_p2/projects/CervicalCancer_
.
├── 001 - Initial Setup.ipynb
├── 002 - Sample Setup.ipynb
├── 003 - Baseline.ipynb
├── 004 - Base CNN Model.ipynb
├── resources
│   ├── TMI_CERVIX_v28n3_2009.pdf
│   ├── Cervix types clasification(1).pdf
│   └── Joe Minichino, Joseph Howse-Learning OpenCV 3 Computer Vision with Python-Packt Publishing (2015).epub
├── reports
├── data
│   ├── train
│   │   ├── Type_1 [250 entries exceeds filelimit, not opening dir]
│   │   ├── Type_2 [781 entries exceeds filelimit, not opening dir]
│   │   └── Type_3 [450 entries exceeds filelimit, not opening dir]
│   ├── test
│   │   └── unknown [512 entries exceeds filelimit, not opening dir]
│   ├── preview
│   ├── results
│   ├── sample
│   │   ├── results
│   │   ├── train
│   │   │   ├── Type_1 [35 entries exceeds filelimit, no

/media/david/5C14F53A14F517AA/code/ana_py27_p2/ana_py27_p2.yml

In [2]:
# A fix for truncated image error
# http://stackoverflow.com/questions/12984426/python-pil-ioerror-image-file-truncated-with-big-images
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
# http://www.pyimagesearch.com/2016/08/08/k-nn-classifier-for-image-classification/
# http://opencv-python-tutroals.readthedocs.io/
#                                 en/latest/py_tutorials/py_ml/py_knn/py_knn_opencv/py_knn_opencv.html#knn-opencv
import cv2
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV

In [4]:
# Get Image Paths

# !!!SAMPLE
train_data_dir = 'data/sample/train' ; validation_data_dir = 'data/sample/valid' ; test_data_dir = 'data/sample/test'
weights_dir = 'data/sample/weights'

# !!!TRAIN
#train_data_dir = 'data/train' ; validation_data_dir = 'data/valid' ; test_data_dir = 'data/test' 
#weights_dir = 'data/weights' ; submission_dir = 'data/submissions'

# Make a list of test_image_paths
test_image_paths = utils.get_non_hidden_dir_contents(os.path.join(test_data_dir, 'unknown'))
print('Test Images')
print(len(test_image_paths))
print(test_image_paths[0:4])


# Make a list of all classified image paths
data_dirs = utils.get_non_hidden_dir_contents(train_data_dir)
data_dirs.extend(utils.get_non_hidden_dir_contents(validation_data_dir))
# Make a nested list and flatten it
full_image_paths = sum([[image_path for image_path in utils.get_non_hidden_dir_contents(data_dir)] 
                       for data_dir in data_dirs],[])
print('Classified Images')
print(len(full_image_paths))
print(full_image_paths[0:4])

Test Images
20
['data/sample/test/unknown/1_142.jpg', 'data/sample/test/unknown/1_531.jpg', 'data/sample/test/unknown/1_921.jpg', 'data/sample/test/unknown/2_1042.jpg']
Classified Images
276
['data/sample/train/Type_1/396.jpg', 'data/sample/train/Type_1/0.jpg', 'data/sample/train/Type_1/102.jpg', 'data/sample/train/Type_1/1024.jpg']


In [5]:
# Make a DataFrame for convenience
data_df = pd.DataFrame(full_image_paths, columns =['path'])
data_df['Type'] = data_df['path'].apply(lambda x: int(x.split('/')[-2].split('_')[1]))

print(data_df.head(2))
print(data_df.tail(2))

# Shuffle the DataFrame before taking a sample
data_df = data_df.sample(frac=1,random_state=random_seed)

# Take sample
#data_df = data_df.head(20)

                               path  Type
0  data/sample/train/Type_1/396.jpg     1
1    data/sample/train/Type_1/0.jpg     1
                                 path  Type
274  data/sample/valid/Type_3/746.jpg     3
275  data/sample/valid/Type_3/811.jpg     3


In [6]:
import cv2
# Split into Train & Test Samples
X = data_df['path'].values
y = data_df['Type'].values
skf = StratifiedKFold(y, shuffle=True, n_folds = 3, random_state=random_seed)
print(X[0:1][0])
print(y[0:4])
hsv = cv2.imread(X[0:1][0])
print(type(X[0:1]))

data/sample/train/Type_3/1275.jpg
[3 1 2 1]
<type 'numpy.ndarray'>


In [7]:
#class_weight_dict = utils.get_class_weight_dict([35, 106, 62])
class_weight_dict = {1: 3.03, 2: 1.0, 3: 1.71}

In [8]:
# http://stackoverflow.com/questions/31259891/put-customized-functions-in-sklearn-pipeline
# Feature extraction function
from sklearn.base import BaseEstimator, TransformerMixin

class Extract_Color_Histogram(BaseEstimator, TransformerMixin):
    def __init__(self, bin_s = 8, size=64):
        self.bin_s = bin_s
        self.size=size
    def transform(self, X, *_):
        """
        Accepts X a numpy array of image paths
        cv2.resize(img, (32,32).flatten()
        
        """
        # To store data as it is being generated
        X_data = []
        for image_path in X:
            img_array = cv2.imread(image_path)#, cv2.COLOR_BGR2HSV)
            # Resize downwards
            img_array = cv2.resize(img_array, (self.size,self.size))
            # Convert to hsv space
            hsv = cv2.cvtColor(img_array, cv2.COLOR_BGR2HSV)
            # Get the hsv histograms
            hist = cv2.calcHist([hsv], [0,1,2], None, (self.bin_s,self.bin_s,self.bin_s), [0, 180, 0, 256, 0,256])
            # Normalize the histograms
            cv2.normalize(hist, hist)
            # Flatten the histograms
            X_data.append(hist.flatten())
        ret_array = np.array(X_data)
        return ret_array
    def fit(self, *_):
        return self

In [9]:
# Create the pipeline
gbc_estimators = []
gbc_estimators.append(('Extract_Color_Histogram', Extract_Color_Histogram()))
gbc_estimators.append(('GBC', GradientBoostingClassifier(loss='deviance',
                                                         learning_rate=0.1,
                                                         n_estimators=100,
                                                         max_depth=3,
                                                         max_features='auto',
                                                         warm_start=True,
                                                         random_state=random_seed)))
gbc_clf = Pipeline(gbc_estimators)

In [10]:
# Use this for preliminary timing on the AWS p2 instance, so can estimate a good size for the sample directory files
import time
start_time = time.time()

In [ ]:
gbc_clf.get_params().keys()

['GBC__max_features',
 'GBC__min_samples_leaf',
 'GBC',
 'GBC__random_state',
 'GBC__init',
 'Extract_Color_Histogram__size',
 'GBC__learning_rate',
 'GBC__subsample',
 'GBC__warm_start',
 'GBC__max_depth',
 'GBC__presort',
 'GBC__loss',
 'Extract_Color_Histogram__bin_s',
 'GBC__verbose',
 'GBC__min_samples_split',
 'steps',
 'Extract_Color_Histogram',
 'GBC__max_leaf_nodes',
 'GBC__min_weight_fraction_leaf',
 'GBC__n_estimators']

In [ ]:
learning_rates=np.array([1.0,0.1,0.01,0.001])
max_features = np.array(['auto','sqrt'])
sizes = np.array([32,64,128])
bins = np.array([8,16,32])
param_grid=dict(GBC__learning_rate=learning_rates,
               Extract_Color_Histogram__size = sizes,
               GBC__max_features=max_features,
               Extract_Color_Histogram__bin_s=bins)
grid = GridSearchCV(estimator=gbc_clf, param_grid=param_grid,n_jobs=-1)
grid.fit(X,y)

In [ ]:
#print(grid.grid_scores_)
print('\n')
print('Best Score: ' , grid.best_score_)
print('\n')
#print(grid.best_estimator_.get_params()['GBC__learning_rate'])
#print(grid.best_estimator_.get_params())
print(grid.best_params_)

In [ ]:
# Use this for preliminary timing on the AWS p2 instance, so can estimate a good size for the sample directory files
print("%f seconds" % (time.time() - start_time))
print('Total Images Processed %d'  % y.shape[0])

In [ ]:
stop

In [ ]:
all_names = ['ETC', 'SVC', 'KNN', 'GNB', 'RCF', 'GBC']
all_results = []
all_results.append(cross_val_score(etc_clf, X, y, cv=skf))
print('Part Completed', all_results[-1])
all_results.append(cross_val_score(svc_clf, X, y, cv=skf))
print('Part Completed', all_results[-1])
all_results.append(cross_val_score(knn_clf, X, y, cv=skf))
print('Part Completed', all_results[-1])
all_results.append(cross_val_score(gnb_clf, X, y, cv=skf))
print('Part Completed', all_results[-1])
all_results.append(cross_val_score(rcf_clf, X, y, cv=skf))
print('Part Completed', all_results[-1])
all_results.append(cross_val_score(gbc_clf, X, y, cv=skf))
print('Part Completed', all_results[-1])

In [ ]:
# Plot Results
fig = plt.figure(figsize=(10,5))
fig.suptitle('Learner Accuracy Scores')
ax = fig.add_subplot(111)
plt.boxplot(all_results)
ax.set_xticklabels(all_names)
plt.show();